## Other interesting features of classes

- Class attributes *vs* object attributes
- Class methods *vs* object methods

Let's create a class having name *Data*, just to store names of objects (*instances* of the class itself). 
We want to keep track of the number of objects that with create in that *Data* class. Such number of objects will be a variable that should be clearly *logically* related to the class itself, and we would like to have a way to translate such *logical relation* into a *structural relation*. 

In what follow
- we define the variable *number_of_obj* within the body of the *Data* class, and we initialize it to the value 0. This is a ***class attribute***: a value that will be *shared* by each object of the class: it is a *global* variable within the namespace of the class itself, so that it can be *seen* by every *method* of the class and its objects; in other words, it becomes an *attribute* of every object. 

- As usual, an \__init\__ method is created as a *constructor* of each newly defined object of the class. Such method asks for a *name* to be associated to the object and stored as an ***object attribute***. Now, as we create a new object, we want to *upgrade* the class variable *number_of_obj*, and this operation should be performed within the \__init\__ method. 

One way to work is:

In [1]:
class Data:
    number_of_obj=0  
    
    def __init__(self, name):
        self.name=name        
        Data.number_of_obj += 1

It *works*:

In [2]:
x1=Data('x1')
x2=Data('x2')

print("Number of objects: ", Data.number_of_obj)
print("Name of object 1: ", x1.name)
print("Name of object 2: ", x2.name)

Number of objects:  2
Name of object 1:  x1
Name of object 2:  x2


Indeed, we created 2 objects (*x1* and *x2*) and such number 2 is correctly recorded in the *Data.number_of_obj* attribute of the class. However this way to proceed is ***not** advised*. In particular, note that the name of the class (*Data*) is *hardcoded* in the definition of the \__init\__ function... if, for some reason, at a later time one decides to change the name of the class, every instance of such name within the body of the class must also be changed.

Much better is to modify the class attibute by using a *method of the class* itself, rather than a *method of the object*. This is the correct way to proceed:

In [3]:
class Data:
    number_of_obj=0  
    
    def __init__(self, name):
        self.name=name        
        self.increment_number()
        
    @classmethod
    def increment_number(cls):
        cls.number_of_obj += 1       

We created a *class method* whose name is *increment_number*, which is preceded by the *@classmethod* ***function decorator***.

Such method refers to the *class* by the name *cls* (when invoked, *cls* will take the name *Data*); it increments the class attribute *cls.number_of_obj* by 1 unit. 

The \__init\__ method of the object invokes such class method by calling *self.increment_number()*: indeed, any *class method* is also a method of each of its instances. Let's try it: 

In [4]:
x1=Data('x1')
x2=Data('x2')

print("Number of objects: ", Data.number_of_obj)
print("Name of object 1: ", x1.name)
print("Name of object 2: ", x2.name)

Number of objects:  2
Name of object 1:  x1
Name of object 2:  x2


Now, suppose you want to keep track of the objects you have instantiated (not only their total number). It would be nice to have a list (*obj_list*) of all such objects. Again, this list should be a *class attribute*: 

In [5]:
class Data:
    number_of_obj=0  
    obj_list=[]
    
    def __init__(self, name):
        self.name=name      
        self.increment_number()
        self.update_list(name)
        
        
    @classmethod
    def increment_number(cls):
        cls.number_of_obj += 1    
        
    @classmethod
    def update_list(cls, name):
        cls.obj_list.append(name)

With the same logic as before, we created a *class method* to properly handle the class attribute *obj_list*; the method is also called in the object *constructor* (\__init\__). Lets'try: 

In [6]:
x1=Data('x1')
x2=Data('x2')

print("Number of objects: ", Data.number_of_obj)
print("List of object names", Data.obj_list)

Number of objects:  2
List of object names ['x1', 'x2']


Suppose now you are interested to create new objects by *summing* other previously defined ones, by assigning names to them, which are defined following some given rule. For instance, from *x1* and *x2*, you want the object *x3=x1+x2* having the name *x1_x2*. Moreover, you want to really use the *operator* '+' to perform the job!

The *dunder* method \__add(self, other)\__ is the way to do it...

In [7]:
class Data:
    number_of_obj=0  
    obj_list=[]
    
    def __init__(self, name):
        self.name=name
        
        self.increment_number()
        self.update_list(name)
        
    def __add__(self, other):     
        new_obj_name=self.name + '_' + other.name
        new_obj=self.join(new_obj_name)
        return new_obj
              
    @classmethod
    def increment_number(cls):
        cls.number_of_obj += 1    
        
    @classmethod
    def update_list(cls, name):
        cls.obj_list.append(name)
        
    @classmethod
    def join(cls, new_name):
        return cls(new_name)
    
    @classmethod
    def reset(cls):
        cls.number_of_obj=0 
        cls.obj_list=[]
        

Some points must be noted here:

- every time the '+' operator will be used on objects of the class *Data*, the corresponding \__add\__ method will be invoked;
- such method produces the appropriate name and stores it in the *new_obj_name* (local) variable;
- \__add\__ calls the *class method* *join*, by passing it *new_obj_name* as argument;
- the class method *join* returns (to the *caller* function \__add\__) a new instance of the class: indeed, *cls(new_name)* is equivalent to *Data(new_name)* which, in turn, is just the creation of a new instance of the *Data* class. This also means that the \__init\__ constructor will be called for the newly created object, so that the *number_of_obj* and the *obj_list* will be properly updated;
- lastly, the \__add\__ method returns the new object.

A class method *reset* was also implemented to reset the class variables to their original values. 

let's see what happen:

In [8]:
x1=Data('x1')
x2=Data('x2')

x3=x1+x2

print("Number of objects: ", Data.number_of_obj)
print("List of object names", Data.obj_list)

Number of objects:  3
List of object names ['x1', 'x2', 'x1_x2']


Note (and try to motivate) the behaviour of the '+' operator in a case like the following:  

In [9]:
Data.reset()

x1=Data('x1')
x2=Data('x2')
x3=Data('x3')

x4=x1+x2+x3

print("Number of objects: ", Data.number_of_obj)
print("List of object names", Data.obj_list)

Number of objects:  5
List of object names ['x1', 'x2', 'x3', 'x1_x2', 'x1_x2_x3']


### Some real dataset

Suppose that we want to read seismic data from INGV files, stored in a given subfolder. Each file contains data corresponding to a specific area. An *info* file in the same subfolder contains *file names* and *area names*. In our case, the *info.dat* is as follow:

```
earthq_sicilia.dat     sicilia
earthq_north.dat       north
earthq_central.dat     central
earthq_emilia.dat      emilia 
```

We add two appropriate class methods to read the data, and one object method to assign attributes to each object:

- *read_info* which asks for a path name (the name of the subfolder) and the name of the *info* file. This method prepares the lists *obj_files* and *obj_names*. Each entry in the *obj_files* list corresponds to the *full* name of the file (the *path* is included). 
- *setup*. This method uses the *pandas* library to get data from the obj_files and, for each *obj* in the *obj_names* list, calls the *set_data* object method;
- *set_data*: assigns the attributes *magnitude* and *depth* to each object.

Note that we have deleted the methods *\__add\__* and *join*, and have modified the \__init\__ one.

One point in the code below deserves particular attention: in the *setup* method you will see a couple of lines of the type

```eval(obj).set_data('magnitude', idata.Magnitude)``` 

the point worth to be noted is the  ``` eval(obj) ``` one: *eval* takes a string and *evaluate* it to the variable having the same name! However, that variable must already exist (more on this in a second) 

In [10]:
import pandas as pd
import numpy as np

class Data:
    number_of_obj=0  
    obj_files=[]
    obj_names=[]
    
    def __init__(self, name):
        self.name=name       
        self.increment_number()
        
    def set_data(self, type_of_data, val):
        
        match type_of_data:
            case 'magnitude':
               self.magnitude=np.array(val)
            case 'depth':
               self.depth=np.array(val)
            case other:
                print(f"{type_of_data} not implemented")                   
              
    @classmethod
    def increment_number(cls):
        cls.number_of_obj += 1    
        
           
    @classmethod
    def read_info(cls, path, info_file):         
        file=path+'/'+info_file
        fi=open(file)
        text=fi.read()
        fi.close()
        text=text.rstrip().splitlines()
        num_lines=len(text)
             
        for line in text:
            line=line.split()
            cls.obj_files.append(path+'/'+line[0])
            cls.obj_names.append(line[1])
            
    @classmethod
    def setup(cls):
        for obj, file in zip(cls.obj_names, cls.obj_files):
            idata=pd.read_csv(file, sep='|')
            idata.rename(columns={"Depth/Km": "Depth"}, inplace=True)
         
            eval(obj).set_data('magnitude', idata.Magnitude)
            eval(obj).set_data('depth', idata.Depth)

In order to use the class just created, we have to 

1. initialize it by reading the *info* file; this is done by calling the class method *Data.read_info*;
2. create the required objects by instantiating the class for each *object name* found in the *Data.obj_names* list;
3. assigning the attributes to the *objects* created at the point (2), by calling *Data.setup* 

Concerning the point (2), the line 

```exec(obj + ' = Data(obj)')``` 

defines a string by concatenating the string *obj* with the string *=Data(obj)*, thereby producing *obj=Data(obj)*, and executes it! The result is the creation of an object of the class for each entry of the *Data.obj_names* list. 

The *Data.setup* class method can now be used as its ``` eval(obj) ``` lines evaluate the strings *obj*'s to variable already defined. 

In [11]:
# Point 1
Data.read_info('L6_data_files', 'info.dat')

# Point 2
for obj in Data.obj_names:
    exec(obj + ' = Data(obj)')

# Point 3
Data.setup()

The names of the *regions* found in the *info.dat* file are now stored in the *Data.obj_name* class variable (which is a list): 

In [12]:
print("region names: ", Data.obj_names)

region names:  ['sicilia', 'north', 'central', 'emilia']


For each region (*object*), magnitude data can be found in the *magnitude object attribute* and accessed by the syntax *obj.magnitude*. For instance, we can enquire about the number of seismic events recorded for the region *sicilia*; this is just the length of the *sicilia.magnitude* numpy array that (as such) has the attribute *size* (length) associated with itself:

In [13]:
print("Number of events for Sicilia: ", sicilia.magnitude.size)

Number of events for Sicilia:  559


The average magnitude of those events is:

In [14]:
print("Average magnitude: ", sicilia.magnitude.mean().round(2))

Average magnitude:  2.33


We can add some functionality through a class *Stat* whose functions are inherited by our Data class. In particular, the *Stat* class implements the *describe* method that summarizes some statistical data about any *object* dataset. 

In [18]:
import pandas as pd
import numpy as np

class Stat():
          
      def set_size(self):
          self.size=len(self.magnitude)
          return self.size
        
      def average(self):
          ave = 0.
          size = self.set_size()
          self.size=size
          for ix in self.magnitude:
              ave=ave+ix
          ave=ave/size
          self.ave=ave
          self.flag=True
          return ave
        
      def standard_deviation(self, force=True):
          if (not self.flag) or (self.flag and force):
             ave=self.average() 
            
          ave=self.ave
          size=self.size
          std=0.
          for ix in self.magnitude:
              std=std+(ix-ave)**2
                
          std=(std/(size-1))**0.5
          self.std=std
          return std
            
      def describe(self):   
              
          self.average()
          self.standard_deviation()
          self.min_mag=np.min(self.magnitude)
          self.max_mag=np.max(self.magnitude)
          self.depth_min=np.min(self.depth)
          self.depth_max=np.max(self.depth)
                    
          print("data-set: %s" % self.name)
          print("Size: %4i" % self.size)
          print("Minimum magnitude:  %5.2f" % self.min_mag)
          print("Maximum magnitude:  %5.2f" % self.max_mag)
          print("Average magnitude:  %5.2f" % self.ave)
          print("Stand. dev:         %5.2f\n" % self.std)
          print("Depths (km):")
          print("Minimum depth: %6.1f, maximum depth %6.1f\n" %
                (self.depth_min, self.depth_max))
                
class Data(Stat):
    number_of_obj=0  
    obj_files=[]
    obj_names=[]
    
    def __init__(self, name):
        self.name=name       
        self.increment_number()
        self.flag=True
        self.size=None
        
    def set_data(self, type_of_data, val):
        
        match type_of_data:
            case 'magnitude':
               self.magnitude=np.array(val)
            
            case 'depth':
               self.depth=np.array(val)
               
            case other:
                print(f"{type_of_data} not implemented")

            
    @classmethod
    def increment_number(cls):
        cls.number_of_obj += 1    
        
           
    @classmethod
    def read_info(cls, path, info_file):         
        file=path+'/'+info_file
        fi=open(file)
        text=fi.read()
        fi.close()
        text=text.rstrip().splitlines()
        num_lines=len(text)
             
        for line in text:
            line=line.split()
            cls.obj_files.append(path+'/'+line[0])
            cls.obj_names.append(line[1])
            
    @classmethod
    def setup(cls):
        for obj, file in zip(cls.obj_names, cls.obj_files):
            idata=pd.read_csv(file, sep='|')
            idata.rename(columns={"Depth/Km": "Depth"}, inplace=True)
            eval(obj).set_data('magnitude', idata.Magnitude)
            eval(obj).set_data('depth', idata.Depth)
            
            

Let's setup the dataset as before:

In [19]:
Data.read_info('L6_data_files', 'info.dat')
for obj in Data.obj_names:
    exec(obj + ' = Data(obj)')
Data.setup()

Now, for instance, we can call *describe* of the *sicilia* object:

In [20]:
sicilia.describe()

data-set: sicilia
Size:  559
Minimum magnitude:   2.00
Maximum magnitude:   4.30
Average magnitude:   2.33
Stand. dev:          0.35

Depths (km):
Minimum depth:    0.0, maximum depth  208.2

